## 문서 분류 모델 학습

### 각종 설정

모델 하이퍼 파라미터(hyper parameter)와 저장 위치 등 설정 정보를 선언

- 구글 드라이브 연동

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)  

Mounted at /gdrive


- 의존성 패키지 설치

In [3]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 57.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2ff6655bbb1c5a12a6eecacf59f23020ff4b941e90286421e1ac185245bee063
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

- TPU 관련 패키지 설치: GPU 사용하면 생략

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/
torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

#### 모델 환경 설정
kcbert-base 모델을 NSMC 데이터로 파인튜닝 설정

In [4]:
from torch.cuda import is_available
import torch 
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/MyDrive/nlpbook/checkpoint-doccls1",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=1,
    #epochs=1,
    #tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

```
- pretrained_model_name : 프리트레인 마친 언어모델의 이름
  * 단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 함

- downstream_corpus_name : 다운스트림 데이터의 이름

- downstream_model_dir : 파인튜닝된 모델의 체크포인트가 저장될 위치.
  * /gdrive/My Drive/nlpbook/checkpoint-doccls: 자신의 구글 드라이브의 내 폴더 하위의 nlpbook/checkpoint-doccls 디렉토리에 모델 체크포인트가 저장

- batch_size : 배치 크기
  * 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 32, TPU라면(torch.cuda.is_available() == False) 4.
  * 코랩 환경에서 TPU는 보통 8개 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기이기 때문에 이렇게 설정

- learning_rate : 러닝레이트. 1회 스텝에서 한 번에 얼마나 업데이트할지에 관한 크기

- max_seq_length : 토큰 기준 입력 문장 최대 길이. 
  * 이보다 긴 문장은 max_seq_length로 자르고, 짧은 문장은 max_seq_length가 되도록 스페셜 토큰(PAD)을 붙여 줌

- epochs : 학습 에폭 수. 3이라면 학습 데이터를 3회 반복 학습

- tpu_cores : TPU 코어 수. 
  * 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available() == True)했다면 0, TPU라면(torch.cuda.is_available() == False) 8

- seed : 랜덤 시드(정수, integer). None을 입력하면 랜덤 시드를 고정하지 않음
```

#### 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정

In [5]:
#args에 지정된 시드로 고정하는 역할
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


#### 로거 설정
메세지 출력 등을 위한 logger를 설정

In [6]:
#각족 로그들을 출력하는 로거를 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoint-doccls1', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/nlpbook/checkpoint-doccls1', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_siz

### 말뭉치 내려받기

- 실습에 사용할 말뭉치(Naver Sentiment Movie review Corpus)를 다운로드
- 코랩 환경 로컬의 root_dir(/content/Korpora)이하에 저장

In [9]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 82.3MB/s]                           
[nsmc] download ratings_test.txt: 4.90MB [00:00, 42.5MB/s]                            


### 토크나이저 준비

- 토큰화를 수행하는 토크나이저를 선언
- kcbert-base 모델이 사용하는 토크나이저를 선언
  * 토크나이저는 토큰화를 수행하는 프로그램
  * 토큰화는 문장을 토큰 시퀀스로 분절하는 과정

In [10]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

### 데이터 전처리

- 딥러닝 모델을 학습하려면 학습 데이터를 배치 단위로 모델에 공급해줘야 함
  * 파이토치에서는  데이터 로더(dataloader)가 수행
  * 데이터 로더: 데이터셋이 보유하고 있는 인스턴스를 배치 크기 만큼 뽑아서(sample) 자료형, 데이터 길이 등 정해진 형식에 맞춰(collate) 배치를 만들어 줌


#### 학습 데이터 구축


- ClassificationDataset: 각 instance를 포함하는 데이터셋 역할을 수행
  * 이 클래스는 NsmcCorpus와 토크나이저를 품고 있음 
  * NsmcCorpus는 CSV 파일 형식의 NSMC 데이터를 “문장(영화 리뷰) + 레이블(긍정, 부정)” 형태로 읽어들이는 역할
  * NsmcCorpus가 넘겨준 데이터(문장, 레이블)를 모델이 학습할 수 있는 형태로 가공
  * 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블 역시 정수(integer)로 바꿔주는 역할


In [11]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 31.595 s]
INFO:ratsnlp:tokenize sentences [took 31.595 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

- input_ids: 인데스로 변환된 토큰 시퀀스
  * 패딩 토큰([PAD])의 인덱스에 해당하는 0이 많이 붙어 있음
  * 분석 대상 문장의 토큰 길이가 max_seq_length보다 짧기 때문
  * 이보다 긴 문장일 경우 128로 줄임

- attention_mask: 해당 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타냄

- token_type_ids는 세그먼트(segment) 정보로 기본값은 모두 0으로 넣음

- label: 정수로 변환된 레이블 정보

- ClassificationDataset은 input_ids, attention_mask, token_type_ids의 길이가 모두 128인 이유
  * 토큰 기준 최대 길이(max_seq_length)를 args에서 128로 설정해 두었기 때문


In [12]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

- 데이터 로더: ClassificationDataset 클래스가 들고 있는 전체 인스턴스 가운데 배치 크기(args의 batch_size)만큼을 뽑아 배치 형태로 가공(nlpbook.data_collator)하는 역할을 수행
  * 데이터 로더는 배치를 만들 때 ClassificationDataset이 들고 있는 전체 인스턴스 가운데 batch_size 갯수만큼을 비복원(replacement=False) 랜덤 추출(Random Sampler)

- sampler: 샘플링 방식을 정의 
- collate_fn: 이렇게 뽑힌 인스턴스를 배치로 만드는 역할을 하는 함수
  * nlpbook.data_collator: 같은 배치에서 인스턴스가 여럿일 때 이를 input_ids, attention_mask 등 종류별로 모으고 파이토치가 요구하는 자료형인 텐서 형태로 바꾸는 역할을 수행

In [14]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,  #배치 크기로 채우지 못한 마지막 불완전 배치의 사용 여부
    num_workers=args.cpu_workers,
)

#### 평가용 데이터 구축

- 학습용 데이터 로더와 달리 평가용 데이터 로더는 SequentialSampler를 사용
- SequentialSampler는 batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할
- 학습 때 배치 구성은 랜덤으로 하는 것이 좋으나 평가할 때는 평가용 데이터 전체를 사용하므로 굳이 랜덤으로 구성할 이유가 없기 때문에 SequentialSampler를 사용

In [15]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

from torch.utils.data import DataLoader, RandomSampler
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(val_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,  #배치 크기로 채우지 못한 마지막 불완전 배치의 사용 여부
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 10.355 s]
INFO:ratsnlp:tokenize sentences [took 10.355 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

### 모델 초기화

- 프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화
- BertForSequenceClassification: 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스


In [16]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

### 모델 학습

Task와 Trainer를 준비

- Task 정의
  * 파이토치 라이트닝이 제공하는 LightningModule 클래스를 상속받아 태스크를 정의
  * 위에서 만든 학습 설정(args)과 준비한 모델(model)을 ClassificationTask에 주입
  * ClassificationTask에는 옵티마이저(optimizer), 러닝 레이트 스케줄러(learning rate scheduler)가 정의되어 있음
  * 옵티마이저는 아담(Adam), 러닝 레이트 스케줄러는 ExponentialLR을 사용

In [17]:

from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

- 트레이너 정의
  * 파이토치 라이트닝 라이브러리의 도움을 받아 GPU/TPU 설정, 로그 및 체크포인트 등의 설정을 알아서 해줌

In [18]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


- 학습
  * 준비한 데이터와 모델로 학습을 시작
  * 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치("/gdrive/MyDrive/nlpbook/checkpoint-doccls1")에 저장

In [20]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]